# Chatper 5.6 ~ 5.8

In [1]:
import numpy as np

## 5.6 Affine/Softmax 계층 구현하기

### 5.6.1 Affine 계층

**Affine 변환(Affine Transformation)** 은 신경망의 순전파 때 수행하는 행렬의 곱입니다.  
즉 **'Affine 계층'** 은 Affine 변환을 수행하는 처리를 의미합니다.


### 5.6.2 배치용 Affine 계층
이번 파트에서는 배치용 Affine 계층을 코드로 구현해보도록 하겠습니다.

In [3]:
class Affine:
    def __init__(self,W,b):
        self.W=W
        self.b=b
        self.x=None
        self.dW=None
        self.db=None
        
    def forward(self,x):
        self.x = x
        out = np.dot(x,self.W) + self.b 
        return out
    
    def backward(self,dout):
        dx = np.dot(dout,self.W.T)
        self.dW = np.dot(self.x,dout)
        self.db = np.sum(dout,axis=0)

        return dx

### 5.6.3 Softmax-With-Loss 계층

> Softmax 함수는 입력 값을 정규화(출력의 합이 1이 되도록 변형)하여 출력합니다.

In [10]:

class SoftmaxWithLoss:
    def __init__(self):
        self.loss=None
        self.y = None
        self.t = None

    def forward(self,x,t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y,self.t)
        
        return loss
    
    def backward(self,dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y-self.t) / batch_size
        return dx

## 5.7 오차역전파법 구현하기

In [14]:
import sys,os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self,input_size,hidden_size,output_size,weight_init_std=.01):
        self.params={}
        self.params['W1'] = weight_init_std * np.random.randn(input_size,hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size,output_size)
        self.params['b2'] = np.zeros(output_size)

        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'],self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'],self.params['b2'])
        
        self.lastlayer = SoftmaxWithLoss()

    def predict(self,x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
    
    def loss(self,x,t):
        y = self.predict(x)
        
        return self.lastlayer.forward(y,t)
    
    def accuracy(self,x,t):
        y = self.predict(x)
        y = np.argmax(y,axis=1)
        if t.ndim != 1:
            np.argmax(t,axis=1)

        accuracy = np.sum(y==t) / float(x.shape[0])

        return accuracy
    
    def numerical_gradient(self,x,t):
        loss_W = lambda W : self.loss(x,t)

        grads={}
        grads['W1'] = numerical_gradient(loss_W,self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W,self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W,self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W,self.params['b2'])

        return grads
    
    def gradient(self,x,t):
        self.loss(x,t)

        dout=1
        dout=self.lastlayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse() # 역전파는 순방향과 반대방향이기 때문에 reverse를 사용해야함 
        for layer in layers:
            dout = layer.backward(dout)

        grads={}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads 

### 기울기 확인 

In [ ]:
import sys,os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet
(x_train,t_train),(x_test,t_test) = load_mnist(normalize=True,one_hot_label=True)

network = TwoLayerNet(input_size=784,hidden_size=50,output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch,t_batch)
grad_backprop = network.gradient(x_batch,t_batch)

for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":"+str(diff))

In [16]:
import sys,os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train,t_train),(x_test,t_test) = load_mnist(normalize=True,one_hot_label=True)
network = TwoLayerNet(input_size=784,hidden_size=50,output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size= 100
learning_rate = 0.1

train_loss_list=[]
train_acc_list =[]
test_acc_list=[]

iter_per_epoch = max(train_size / batch_size,1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size,batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    grads = network.gradient(x_batch,t_batch)

    for key in ('W1','b1','W2','b2'):
        network.params[key] -= learning_rate * grads[key]

    loss = network.loss(x_batch,t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train,t_train)
        test_acc = network.accuracy(x_test,t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)

        print(train_acc,test_acc)

/var/folders/6b/vq51j9_153n_8c0d82cvxh7w0000gn/T/ipykernel_5233/3430132595.py:40: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  accuracy = np.sum(y==t) / float(x.shape[0])


0.0 0.0


KeyboardInterrupt: 